In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  
model = GPT2LMHeadModel.from_pretrained('gpt2')

/home/gini/sylve057/.conda/envs/llm_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/gini/sylve057/.conda/envs/llm_env/lib/python3.9/site-packages/huggingface_hub/file_download.py:1003: UserWarning: Not enough free disk space to download the file. The expected file size is: 548.11 MB. The target location /home/gini/sylve057/.cache/huggingface/hub only has 520.14 MB free disk space.
  warnings.warn(
/home/gini/sylve057/.conda/envs/llm_env/lib/python3.9/site-packages/huggingface_hub/file_download.py:1003: UserWarning: Not enough free disk space to download the file. The expected file size is: 548.11 MB. The target location /home/gini/sylve057/.cache/huggingface/hub/models--gpt2/blobs only has 520.14 MB free disk space.
  warnings.warn(


In [6]:
input_text = "Hi?"

input_ids = tokenizer.encode(input_text, return_tensors='pt')

output = model.generate(input_ids, max_length=100)

print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hi?

I'm not sure if you're aware of the fact that the US government has been using the term "global warming" to describe the global warming that is happening in the US.

The US government has been using the term "global warming" to describe the global warming that is happening in the US.

The US government has been using the term "global warming" to describe the global warming that is happening in the US.

The US government has been using
